# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_page (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


## Connect to Spark

In [1]:
import findspark
findspark.init()

from pyspark.sql.window import Window
from pyspark.sql.context import SQLContext
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, functions as F

import json
from itertools import groupby

# Set configuration
conf = SparkConf().setMaster('local').setAppName('Clickstream Analysis')

# Creating classes to interact with Spark
sc = SparkContext(conf=conf)
se = SparkSession(sc)
sql = SQLContext(sc)

sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-04-24 14:59:58,989 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local appName=Clickstream Analysis>

In [2]:
# Read clickstream.csv
df = sql.read.csv('hdfs:/data/clickstream.csv', sep='\t', header=True, inferSchema=True)
df.createOrReplaceTempView('clickstream')

# Show first 5 rows
df.show(5)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



## Spark SQL solution

In [3]:
# SQL query
query = """
    WITH min_timestamps AS (
        SELECT
            user_id, session_id, MIN(CAST(timestamp AS INT)) AS min_ts
        FROM clickstream
        WHERE event_type LIKE "%error%"
        GROUP BY user_id, session_id
    ),
    filtered AS (
        SELECT c.user_id, c.session_id, event_page, timestamp
        FROM clickstream c
        LEFT JOIN min_timestamps t
        ON c.user_id = t.user_id AND c.session_id = t.session_id
        WHERE event_type = "page" AND (min_ts IS NULL OR timestamp < min_ts)
    ),
    windowed AS (
        SELECT
            user_id,
            session_id,
            event_page,
            timestamp,
            LAG(event_page) OVER (
              PARTITION BY user_id, session_id
              ORDER BY timestamp
            ) AS prev_event_page
        FROM filtered
    ),
    routes AS (
        SELECT
            user_id,
            session_id,
            CONCAT_WS('-', COLLECT_LIST(event_page)) AS route
        FROM windowed
        WHERE prev_event_page IS NULL OR event_page != prev_event_page
        GROUP BY user_id, session_id
    )
    SELECT
        route,
        COUNT(*) as count
    FROM (SELECT DISTINCT * FROM routes)
    GROUP BY route
    ORDER BY count DESC
    LIMIT 30
"""

# Show top 30 routes
top_30_routes = se.sql(query)
top_30_routes.show(30, truncate=False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8184 |
|main-archive         |1113 |
|main-rabota          |1047 |
|main-internet        |897  |
|main-bonus           |870  |
|main-news            |769  |
|main-tariffs         |677  |
|main-online          |587  |
|main-vklad           |518  |
|main-rabota-archive  |170  |
|main-archive-rabota  |167  |
|main-bonus-archive   |143  |
|main-rabota-bonus    |139  |
|main-bonus-rabota    |135  |
|main-news-rabota     |135  |
|main-archive-internet|132  |
|main-rabota-news     |130  |
|main-internet-rabota |129  |
|main-archive-news    |126  |
|main-rabota-internet |124  |
|main-internet-archive|122  |
|main-archive-bonus   |117  |
|main-internet-bonus  |115  |
|main-tariffs-internet|114  |
|main-news-archive    |113  |
|main-news-internet   |109  |
|main-archive-tariffs |104  |
|main-internet-news   |103  |
|main-tariffs-archive |103  |
|main-rabota-main     |94   |
+---------

## Spark RDD solution

In [4]:
rdd = df.rdd

# Groupes consecutive duplicates
def windowed(obj):
    sorted_pages = sorted(obj[1], key=lambda row: row[1])
    sorted_pages = map(lambda row: row[0], sorted_pages)
    return (obj[0], [key for key, _ in groupby(sorted_pages)])

# Get timestamps of error sessions
min_timestamps = rdd.filter(lambda row: 'error' in row.event_type)\
                    .map(lambda row: ((row.user_id, row.session_id), row.timestamp))\
                    .reduceByKey(min)

# Get filtered sessions
filtered = rdd.filter(lambda row: row.event_type == 'page')\
              .map(lambda row: ((row.user_id, row.session_id), (row.event_page, row.timestamp)))\
              .leftOuterJoin(min_timestamps)\
              .filter(lambda row: row[1][1] is None or (row[1][0][1] < row[1][1]))\
              .map(lambda row: (row[0], row[1][0]))\
              .filter(lambda row: row[1][1] is not None)

# Get distinct routes
routes = filtered.groupByKey()\
                 .map(windowed)\
                 .map(lambda row: (row[0], '-'.join(list(row[1]))))
route_distinct = routes.map(lambda row: (row[1], 1)).reduceByKey(lambda a, b: a + b)

# Get top 30 routes
top_30_routes = route_distinct.takeOrdered(30, key=lambda row: -row[1])
top_30_routes

[('main', 8184),
 ('main-archive', 1113),
 ('main-rabota', 1047),
 ('main-internet', 897),
 ('main-bonus', 870),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 143),
 ('main-rabota-bonus', 139),
 ('main-news-rabota', 135),
 ('main-bonus-rabota', 135),
 ('main-archive-internet', 132),
 ('main-rabota-news', 130),
 ('main-internet-rabota', 129),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 115),
 ('main-tariffs-internet', 114),
 ('main-news-archive', 113),
 ('main-news-internet', 109),
 ('main-archive-tariffs', 104),
 ('main-internet-news', 103),
 ('main-tariffs-archive', 103),
 ('main-rabota-main', 94)]

## Spark DF solution

In [5]:
# Get timestamps of error sessions
min_timestamps = df.select('user_id', 'session_id', 'timestamp')\
                   .filter(df.event_type.like('%error%'))\
                   .groupBy('user_id', 'session_id')\
                   .agg(F.min('timestamp').alias('min_ts'))

# Get filtered sessions
filtered = df.join(min_timestamps, ['user_id', 'session_id'], 'left')\
             .filter((min_timestamps.min_ts.isNull()) | (df.timestamp < min_timestamps.min_ts))\
             .filter(df.event_type == 'page')\
             .select('user_id', 'session_id', 'timestamp', 'event_page')

# Groupes consecutive duplicates
windowed = filtered.withColumn('prev_event_page',
                              F.lag('event_page', 1).over(Window.partitionBy('user_id', 'session_id').orderBy('timestamp')))\
                   .filter((F.col('prev_event_page').isNull()) | (F.col('event_page') != F.col('prev_event_page')))

# Get distinct routes
routes = windowed.groupBy('user_id', 'session_id')\
                 .agg(F.collect_list(df.event_page).alias('pages'))\
                 .groupBy('user_id', 'session_id', 'pages')\
                 .agg(F.array_join('pages', '-').alias('route'))\
                 .select('route')
route_distinct = routes.groupBy('route')\
                       .agg(F.count('*').alias('count'))\
                       .orderBy(F.desc('count'))

# Get top 30 routes
top_30_routes = route_distinct.limit(30).collect()
top_30_routes = {k: v for k,v in top_30_routes}
top_30_routes

{'main': 8184,
 'main-archive': 1113,
 'main-rabota': 1047,
 'main-internet': 897,
 'main-bonus': 870,
 'main-news': 769,
 'main-tariffs': 677,
 'main-online': 587,
 'main-vklad': 518,
 'main-rabota-archive': 170,
 'main-archive-rabota': 167,
 'main-bonus-archive': 143,
 'main-rabota-bonus': 139,
 'main-news-rabota': 135,
 'main-bonus-rabota': 135,
 'main-archive-internet': 132,
 'main-rabota-news': 130,
 'main-internet-rabota': 129,
 'main-archive-news': 126,
 'main-rabota-internet': 124,
 'main-internet-archive': 122,
 'main-archive-bonus': 117,
 'main-internet-bonus': 115,
 'main-tariffs-internet': 114,
 'main-news-archive': 113,
 'main-news-internet': 109,
 'main-archive-tariffs': 104,
 'main-internet-news': 103,
 'main-tariffs-archive': 103,
 'main-rabota-main': 94}

### Save and Submit result

In [6]:
with open('result.json', 'w') as f:
    top_10_routes = dict(list(top_30_routes.items())[:10])
    f.write(json.dumps(top_10_routes))

In [ ]:
!curl -F file=@result.json 51.250.120.121:80/MDS-LSML1/apoletaev/w6/2

In [9]:
sc.stop()